In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [17]:
import numpy as np
import magnificationbias_covariance
import gglens_x_ggclustering_likelihood as utils
import os, json, copy
import matplotlib.pyplot as plt
from my_python_package.utility import silencer
plt.rcParams['font.family'] = 'serif'# 'sans-serif'
plt.rcParams['font.serif'] = "STIXGeneral"
plt.rcParams['font.size'] = 25
plt.rcParams['axes.linewidth'] = 1.5
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['xtick.minor.visible'] = True
plt.rcParams['ytick.minor.visible'] = True
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['figure.figsize'] = (10,6)

plt.rc("text",usetex=True)
plt.rc("font",family="serif")
plt.rc("font",serif="STIXGeneral")

In [7]:
mc = magnificationbias_covariance.magnificationbias_covariance()
config = utils.config('config_mock_fiducial.json', use_interp_meas_corr=False)
info = {}
info['zl_list'] = config.parameters['observables']['lensing']['redshift']
info['zs'] = 1.234
info['galaxy_list'] = [1.78, 2.12, 2.28]

Omega_s = (136.9*deg2rad(1.0)**2)
info['Omega_s'] = Omega_s
info['alpha_list'] = [2.259,3.563,3.729]
info['shape_noise'] = 0.2207**2
n_s = 7.9549 # arcmin^-2
info['n_s'] = n_s / deg2rad(1.0/60.0)**2

x,y,cov = config.load_data()
n = config.probes.n_redshift['lensing']
cov_lens = cov.get_covariance('lensing')
R = x.get_radial_bin('lensing', 0)
dlogR = np.diff(np.log10(R))[0]

initialize cosmo_class
Initialize pklin emulator
set up cosmology :  [[ 0.02225  0.1198   0.6844   3.094    0.9645  -1.     ]]
set up cosmology :  [[ 0.02254  0.11417  0.721    3.094    0.97    -1.     ]]
0.020932578062250386
set up cosmology :  [[ 0.02254     0.11417     0.721       3.11493258  0.97       -1.        ]]


In [9]:
#test setups
Rcombinations = [[80.0, 80.0], [40.0, 40.0], [10.0, 10.0], [6.0, 6.0], [3.0,3.0],
                 [80.0, 40.0], [80.0, 10.0], [80.0, 6.0], [80.0, 3.0], 
                 [40.0, 10.0], [40.0, 6.0], [40.0, 3.0], 
                 [10.0, 6.0] , [10.0, 3.0]]
zcombinations = [[0,0], [1,1], [2,2],
                 [0,1], [0,2],
                 [1,2]]

In [84]:
# hyper parameter
# l : min, max, N
# k : min, max, N
# dump : dump_peak_ratio

def check(hyper_param_dict):
    if 'l' in hyper_param_dict.keys():
        mc.l = hyper_param_dict['l']
    if 'k' in hyper_param_dict.keys():
        mc.k = hyper_param_dict['k']
    dump_peak_ratio = hyper_param_dict.get('dump_peak_ratio', 1e2)
    s = silencer()
    s.start()
    mc.prepare_dcov(info)
    s.end()
    
    dR = np.array([10**(-dlogR/2.0),10**(dlogR/2.0)])
    dcov = []
    cond = []
    for i, j in zcombinations:
        for r1, r2 in Rcombinations:
            R1, R2 = r1*dR, r2*dR
            dc = mc.get_dcov(i, j, R1, R2, show_integrand=False, dump_peak_ratio=dump_peak_ratio)
            cond.append([i,j,r1,r2])
            dcov.append(dc)
    return np.array(dcov), cond

### N of l dependency

In [19]:
h = {'l':np.logspace(-1, 4, 1000)}
dcov, cond = check(h)

In [21]:
h = {'l':np.logspace(-1, 4, 10000)}
dcov2, cond = check(h)

In [26]:
h = {'l':np.logspace(-1, 4, 100000)}
dcov3, cond = check(h)

In [28]:
print('max of (l=1e3)/(l=1e5)', np.max(abs(1.0 - dcov/dcov3)))
print('max of (l=1e4)/(l=1e5)', np.max(abs(1.0 - dcov2/dcov3)))

max of (l=1e3)/(l=1e5) 0.00037730843223893196
max of (l=1e4)/(l=1e5) 1.2552873661642039e-06


### min of l dependency

In [33]:
h = {'l':np.logspace(-2, 4, 5000)}
dcov, cond = check(h)

In [34]:
h = {'l':np.logspace(-1.5, 4, 5000)}
dcov2, cond = check(h)

In [35]:
h = {'l':np.logspace(-1, 4, 5000)}
dcov3, cond = check(h)

In [36]:
print('max of (lmin=-2)  /(lmin=-1)', np.max(abs(1.0 - dcov/dcov3)))
print('max of (lmin=-1.5)/(lmin=-1)', np.max(abs(1.0 - dcov2/dcov3)))

max of (lmin=-2)  /(lmin=-1) 0.000490431610822184
max of (lmin=-1.5)/(lmin=-1) 0.00044242509686998055


### max of l dependency

In [37]:
h = {'l':np.logspace(-1, 5, 5000)}
dcov, cond = check(h)

In [38]:
h = {'l':np.logspace(-1, 4.5, 5000)}
dcov, cond = check(h)

In [39]:
h = {'l':np.logspace(-1, 4, 5000)}
dcov3, cond = check(h)

In [40]:
print('max of (lmax=5)  /(lmax=4)', np.max(abs(1.0 - dcov/dcov3)))
print('max of (lmax=4.5)/(lmax=4)', np.max(abs(1.0 - dcov2/dcov3)))

max of (lmax=5)  /(lmax=4) 0.0006319720787808514
max of (lmax=4.5)/(lmax=4) 0.00044242509686998055


### N of k dependency

In [50]:
h = {'k':np.logspace(-5, 2, 200)}
dcov, cond = check(h)

In [51]:
h = {'k':np.logspace(-5, 2, 400)}
dcov2, cond = check(h)

In [54]:
h = {'k':np.logspace(-5, 2, 800)}
dcov3, cond = check(h)

In [55]:
h = {'k':np.logspace(-5, 2, 2000)}
dcov4, cond = check(h)

In [57]:
print('max of (N=200)/(N=2000)', np.max(abs(1.0 - dcov/dcov4)))
print('max of (N=400)/(N=2000)', np.max(abs(1.0 - dcov2/dcov4)))
print('max of (N=800)/(N=2000)', np.max(abs(1.0 - dcov3/dcov4)))

max of (N=200)/(N=2000) 0.02119630453339072
max of (N=400)/(N=2000) 0.005079140877372157
max of (N=800)/(N=2000) 0.0011063523291193755


### min of k dependency

In [58]:
h = {'k':np.logspace(-3, 2, 800)}
dcov, cond = check(h)

In [59]:
h = {'k':np.logspace(-4, 2, 800)}
dcov2, cond = check(h)

In [60]:
h = {'k':np.logspace(-5, 2, 800)}
dcov3, cond = check(h)

In [61]:
print('max of (kmin=-3)/(kmin=-5)', np.max(abs(1.0 - dcov/dcov3)))
print('max of (kmin=-4)/(kmin=-5)', np.max(abs(1.0 - dcov2/dcov3)))

max of (kmin=-3)/(kmin=-5) 0.000644423069950939
max of (kmin=-4)/(kmin=-5) 0.00035803368936782043


### max of k dependency

In [66]:
h = {'k':np.logspace(-5, 1, 800)}
dcov, cond = check(h)

In [67]:
h = {'k':np.logspace(-5, 3, 800)}
dcov2, cond = check(h)

In [68]:
h = {'k':np.logspace(-5, 2, 800)}
dcov3, cond = check(h)

In [69]:
print('max of (kmax=1)/(kmax=2)', np.max(abs(1.0 - dcov/dcov3)))
print('max of (kmax=3)/(kmax=2)', np.max(abs(1.0 - dcov2/dcov3)))

max of (kmax=1)/(kmax=2) 0.0005214634175170207
max of (kmax=3)/(kmax=2) 0.000398801550788197


### dump dependency

In [85]:
h = {'dump_peak_ratio':10**2.5}
dcov, cond = check(h)

In [86]:
h = {'dump_peak_ratio':10**3}
dcov2, cond = check(h)

In [87]:
h = {'dump_peak_ratio':10**4}
dcov3, cond = check(h)

In [90]:
h = {'dump_peak_ratio':1e2}
dcov4, cond = check(h)

In [91]:
print('max of (d=1e2.5)/(d=1e2)', np.max(abs(1.0 - dcov/dcov4)))
print('max of (d=1e3)  /(d=1e2)', np.max(abs(1.0 - dcov2/dcov4)))
print('max of (d=1e4)  /(d=1e2)', np.max(abs(1.0 - dcov3/dcov4)))

max of (d=1e2.5)/(d=1e2) 0.0014671478991530318
max of (d=1e3)  /(d=1e2) 0.001615317041046671
max of (d=1e4)  /(d=1e2) 0.0016316448394468885


## conclusion
The choice of hyper parameters
- k = np.array(-5, 2, 400)
- l = np.array(-1, 4, 1000)
- dump_peak_ratio = 1e2

is good to get precise covariance matrix at sub percent level.

In [1]:
import os

In [3]:
os.path.dirname('aaa')

''